In [165]:
from IPython.display import display, HTML

html_code = """
<canvas id="world" width="1200" height="400" style="background:#0b0b10; display:block; margin:0 auto;"></canvas>

<div style="text-align:center; margin-top:6px; color:#ddd; font-family:Arial, sans-serif;">
  <label style="margin-right:10px">contraction amplitude
    <input id="alpha" type="range" min="0" max="0.6" step="0.01" value="0.25">
  </label>
  <label style="margin-right:10px">contraction speed
    <input id="speed" type="range" min="0.005" max="0.08" step="0.005" value="0.02">
  </label>
  <label style="margin-right:10px">lift factor
    <input id="lift" type="range" min="0" max="1" step="0.05" value="0.9">
  </label>
  <label style="margin-right:10px">spring K
    <input id="k" type="range" min="0.02" max="0.6" step="0.01" value="0.15">
  </label>
  <button id="regen" style="margin-left:10px">regen graph</button>
</div>

<script>
(function(){
  if (window.creatureLoop) cancelAnimationFrame(window.creatureLoop);

  const canvas = document.getElementById("world");
  const ctx = canvas.getContext("2d");

  let contractionAlpha = parseFloat(document.getElementById('alpha').value);
  let contractionSpeed = parseFloat(document.getElementById('speed').value);
  let liftFactor = parseFloat(document.getElementById('lift').value);
  let springK = parseFloat(document.getElementById('k').value);

  document.getElementById('alpha').oninput = e => contractionAlpha = parseFloat(e.target.value);
  document.getElementById('speed').oninput = e => contractionSpeed = parseFloat(e.target.value);
  document.getElementById('lift').oninput = e => liftFactor = parseFloat(e.target.value);
  document.getElementById('k').oninput = e => springK = parseFloat(e.target.value);
  document.getElementById('regen').onclick = () => buildConnectedGraph();

  const dt = 0.6;
  const globalDamping = 0.995;
  const baseFrictionHigh = 0.9;
  const baseFrictionLow = 0.1;

  class Node {
    constructor(x,y){
      this.x = x; this.y = y;
      this.vx = 0; this.vy = 0;
      this.radius = 6;
      this.friction = baseFrictionHigh;
    }
  }

  class Edge {
    constructor(a,b){
      this.a = a; this.b = b;
      this.phase = 0;
      this.contracting = false;
      this.lift = null; // oriented contraction {stay,lift}
      this.rest = 0;
    }
  }

  function dist(n1,n2){ return Math.sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2); }

  let nodes = [];
  let edges = [];
  const N_DEFAULT = 5;
  const EXTRA_EDGES = 3;

  function edgeExists(a,b){
    for(let e of edges) if((e.a===a&&e.b===b)||(e.a===b&&e.b===a)) return true;
    return false;
  }

  function buildConnectedGraph(N=N_DEFAULT){
    nodes = [];
    edges = [];
    for(let i=0;i<N;i++){
      let angle = (i/N)*Math.PI*2 + (Math.random()-0.5)*0.5;
      let r = 80 + Math.random()*60;
      let cx = canvas.width/2 + Math.cos(angle)*r;
      let cy = canvas.height/2 + Math.sin(angle)*r;
      nodes.push(new Node(cx + (Math.random()-0.5)*40, cy + (Math.random()-0.5)*40));
    }
    for(let i=1;i<N;i++){
      let j = Math.floor(Math.random()*i);
      let e = new Edge(i,j);
      e.rest = dist(nodes[i], nodes[j]);
      edges.push(e);
    }
    let tries=0;
    while(edges.length < (N-1+EXTRA_EDGES) && tries<200){
      let a = Math.floor(Math.random()*N);
      let b = Math.floor(Math.random()*N);
      if(a!==b && !edgeExists(a,b)){
        let e = new Edge(a,b);
        e.rest = dist(nodes[a], nodes[b]);
        edges.push(e);
      }
      tries++;
    }
  }

  buildConnectedGraph();

  let trail = [];
  const maxTrail = 20000;

  // --- Python / JS API Controls ---
  window.contractEdge = function(nodeStay, nodeLift){
    let e = edges.find(e => (e.a===nodeStay && e.b===nodeLift) || (e.a===nodeLift && e.b===nodeStay));
    if(e){
      e.contracting = true;
      // fix orientation to edge nodes
      if(e.a === nodeStay || e.b === nodeStay){
        e.lift = {stay: nodeStay, lift: nodeLift};
      } else {
        // fallback, keep original argument orientation
        e.lift = {stay: nodeStay, lift: nodeLift};
      }
      e.phase = 0;
    }
  };

  window.growNode = function(baseIndex){
    let base = nodes[baseIndex];
    let newNode = new Node(base.x + Math.random()*40-20, base.y + Math.random()*40-20);
    nodes.push(newNode);
    let e = new Edge(baseIndex, nodes.length-1);
    e.rest = dist(base, newNode);
    edges.push(e);
  };

  window.addEdge = function(i,j){
    if(!edgeExists(i,j)){
      let e = new Edge(i,j);
      e.rest = dist(nodes[i], nodes[j]);
      edges.push(e);
    }
  };

  function updatePhysics(){
    // update contractions
    for(let e of edges){
      if(e.contracting){
        e.phase += contractionSpeed;
        if(e.phase>=1){ e.phase=1; e.contracting=false; e.lift=null; }
      } else { e.phase -= contractionSpeed; if(e.phase<0) e.phase=0; }
    }

    // update node friction
    for(let i=0;i<nodes.length;i++){
      let n = nodes[i];
      let maxPhase=0;
      for(let e of edges) if(e.a===i||e.b===i) if(e.phase>maxPhase) maxPhase=e.phase;
      n.friction = baseFrictionHigh*(1-liftFactor*maxPhase) + baseFrictionLow*(liftFactor*maxPhase);
    }

    // spring forces
    for(let e of edges){
    let n1 = nodes[e.a], n2 = nodes[e.b];
    let dx = n2.x - n1.x;
    let dy = n2.y - n1.y;
    let d = Math.sqrt(dx*dx + dy*dy) + 1e-8;
    let L = e.rest*(1 - contractionAlpha*e.phase);
    let F = springK*(d - L);
    
    if(e.lift){
        let stay = nodes[e.lift.stay];
        let lift = nodes[e.lift.lift];
        let dxl = lift.x - stay.x;
        let dyl = lift.y - stay.y;
        let dl = Math.sqrt(dxl*dxl + dyl*dyl) + 1e-8;
        let fxl = F * dxl / dl;
        let fyl = F * dyl / dl;
        lift.vx -= fxl;   // move lift toward stay
        lift.vy -= fyl;
        stay.vx *= 0.99;  // optional slight damping
        stay.vy *= 0.99;
    } else {
        // normal spring
        let fx = F*dx/d, fy = F*dy/d;
        n1.vx += fx*0.5; n1.vy += fy*0.5;
        n2.vx -= fx*0.5; n2.vy -= fy*0.5;
    }
}


    // update node positions
    for(let n of nodes){
      n.vx *= n.friction; n.vy *= n.friction;
      n.vx *= globalDamping; n.vy *= globalDamping;
      n.x += n.vx*dt; n.y += n.vy*dt;
    }

    // update trail
    let cx = nodes.reduce((s,n)=>s+n.x,0)/nodes.length;
    let cy = nodes.reduce((s,n)=>s+n.y,0)/nodes.length;
    trail.push({x:cx,y:cy});
    if(trail.length>maxTrail) trail.shift();
  }

  function draw(){
    ctx.clearRect(0,0,canvas.width,canvas.height);

    // trail
    ctx.strokeStyle="white";
    ctx.beginPath();
    for(let i=0;i<trail.length;i++){
      let p=trail[i];
      if(i===0) ctx.moveTo(p.x,p.y); else ctx.lineTo(p.x,p.y);
    }
    ctx.stroke();

    // edges
    for(let e of edges){
      let n1=nodes[e.a], n2=nodes[e.b];
      ctx.strokeStyle = e.phase>0?"red":"gray";
      ctx.lineWidth = 2;
      ctx.beginPath();
      ctx.moveTo(n1.x,n1.y);
      ctx.lineTo(n2.x,n2.y);
      ctx.stroke();
    }

    // nodes
    for(let n of nodes){
      let f=(n.friction-baseFrictionLow)/(baseFrictionHigh-baseFrictionLow);
      f=Math.max(0,Math.min(1,f));
      let r=30*(1-f)+20*f;
      let g=200*f+80*(1-f);
      let b=200*(1-f)+120*f;
      ctx.fillStyle=`rgb(${r},${g},${b})`;
      ctx.beginPath();
      ctx.arc(n.x,n.y,n.radius,0,Math.PI*2);
      ctx.fill();

      ctx.fillStyle="#222";
      ctx.fillRect(n.x-10,n.y+10,20,4);
      ctx.fillStyle="#fff";
      ctx.fillRect(n.x-10,n.y+10,20*f,4);
    }

    // mass center
    let cx = nodes.reduce((s,n)=>s+n.x,0)/nodes.length;
    let cy = nodes.reduce((s,n)=>s+n.y,0)/nodes.length;
    ctx.fillStyle="white";
    ctx.beginPath();
    ctx.arc(cx,cy,3,0,Math.PI*2);
    ctx.fill();
  }

  function loop(){
    updatePhysics();
    draw();
    window.creatureLoop = requestAnimationFrame(loop);
  }

  loop();

})();

</script>
"""

display(HTML(html_code))


In [199]:
# # Contract edge from node 0 to node 1
# display(Javascript("growNode(0);"))
display(Javascript("contractEdge(0,3);"))
# display(Javascript("addEdge(0,3);"))

<IPython.core.display.Javascript object>

In [168]:
# Contract edge from node 0 to node 1
display(Javascript("contractEdge(0,1);"))

# Grow a node connected to node 2
display(Javascript("growNode(2);"))

# Add edge between node 3 and node 5
display(Javascript("addEdge(3,5);"))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [149]:
# Grow a new node from node 0
# send_controls([{"action":"grow","node":0}])

# Add edge between 1 and 2
# send_controls([{"action":"add_edge","edge":[1,2]}])

# # Contract edge 1->2 (1 stays, 2 lifts)
send_controls([{"action":"contract","edge":[1,2]}])


<IPython.core.display.Javascript object>

In [80]:
send_controls([{"action":"contract","edge":[0,1]}])

<IPython.core.display.Javascript object>